In [1]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig,
)
import geopandas as gpd

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

c:\Users\Pavel\Documents\GitHub\runaway-satellite-detection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
resolution = 10
config = SHConfig()

In [3]:
gdf = gpd.read_file('train/pac_2024_training.shp')

In [4]:
def get_bbox(coords):
    coords = np.array(coords)
    x_coords, y_coords = coords[:, 0], coords[:, 1]

    # Calculate the center of the polygon
    center_x, center_y = np.mean(x_coords), np.mean(y_coords)

    # Calculate the bounding box
    min_x, max_x = np.min(x_coords), np.max(x_coords)
    min_y, max_y = np.min(y_coords), np.max(y_coords)

    # Calculate the current width and height of the bounding box
    width, height = max_x - min_x, max_y - min_y

    # Expand the square 5 times
    half_width, half_height = 2.5 * width, 2.5 * height

    # Calculate the top-left and bottom-right points
    top_left = (center_x - half_width, center_y + half_height)
    bottom_right = (center_x + half_width, center_y - half_height)

    return BBox(bbox=[top_left[0], top_left[1], bottom_right[0], bottom_right[1]], crs=CRS.WGS84)

In [5]:
def get_image(coords, time):
    bbox = get_bbox(coords)
    size = bbox_to_dimensions(bbox, resolution=resolution)
    evalscript = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04", "B08"],
                units: "DN"
            }],
            output: {
                bands: 4,
                sampleType: "AUTO"
            }
        }
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02, sample.B08];
    }
    """
    return SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time,
            )
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF)
        ],
        bbox=bbox,
        size=size,
        config=config
    )

In [6]:
for idx, row in gdf.iterrows():
    coords = row['geometry'].coords

    year = row['yr']
    start = datetime.datetime(year - 1, 9, 1)
    end = datetime.datetime(year + 1, 5, 31)
    n_chunks = 21
    tdelta = (end - start) / n_chunks
    edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
    slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

    list_of_requests = []
    for slot in slots:
        list_of_requests.append(get_image(coords, slot))
    list_of_requests = [request.download_list[0] for request in list_of_requests]

    data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=20, show_progress=True)

    break

  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: Configuration parameters 'sh_client_id' and 'sh_client_secret' have to be set in order to authenticate with Sentinel Hub service. Check https://sentinelhub-py.readthedocs.io/en/latest/configure.html for more info.